# 利用 GEOHASH 算法的第三方库 pygeohash 实现空间分块索引

In [128]:
import geojson
import numpy as np
import pygeohash as pgh
import matplotlib.pyplot as matplt



## 导入 Json 文件，提出 Multipolygon List

In [129]:
vbList=[[[]]]
with open('../data/victoria.geojson') as vbJson:
    vbList = geojson.load(vbJson).geometries[0].coordinates
    

## 分立存储 polygon

In [130]:
num_polygon = len(vbList)
# polygons: vbList[i][0]=[[]], i in range(num_polygon)

polygon_cdnts = []

for i in range(num_polygon):
    polygon_cdnts.append(vbList[i][0])

## 对一个 polygon ，分离 lon, lat, idx 列表

In [131]:
polygon_lat = []
polygon_lon = []
polygon_idx = []
for i in range(len(polygon_cdnts)):
    xList = []
    yList = []
    pList = []
    for j in range(len(polygon_cdnts[i])):
        xList.append(polygon_cdnts[i][j][0])
        yList.append(polygon_cdnts[i][j][1])
        pList.append(pgh.encode(yList[j], xList[j]))
    polygon_lat.append(yList)
    polygon_lon.append(xList)
    polygon_idx.append(pList)


In [140]:
## 单块 polygon 地图画布 (board) 精细分域与反编码

def div_board(latlist, lonlist):
    # 获取最值
    latmax = np.max(latlist)
    latmin = np.min(latlist)    
    lonmax = np.max(lonlist)
    lonmin = np.min(lonlist)
    # 枚举划分/步长：
    accu = 0.1
    board_lat = []
    board_lon = []
    board_idx = []
    # 存储所有划分的出来的区域的编码
    for i in np.arange(latmin, latmax, accu):
        for j in np.arange(lonmin, lonmax, accu):
            c = pgh.encode(i, j)
            a,b = pgh.decode(c)
            board_lat.append(a)
            board_lon.append(b)
            board_idx.append(c)
    return board_lon, board_lat, board_idx



In [ ]:
def get_geolonlat_zt(total):
    new_total=[]  # 全部的多级编码
    block_dict = {}
    central_dict = {}
    for i in total:     # 得到的前n位全部的编码
        new_total.append(i[:5])  # 只能取4个
    new_total = list(set(new_total))  # 得到编码的个数
    new_total_lonlat = []
    for block in new_total:
        lat_zt, lon_zt = get_lonlat(block)
        new_total_lonlat.append([lon_zt, lat_zt])
    return new_total_lonlat

def match_zt(total):
    new_total_lonlat = get_geolonlat_zt(total)
    all_centralpoint = new_total_lonlat
    new_total_lon = []
    new_total_lat = []
    for point in all_centralpoint:
        new_total_lon.append(point[0])
        new_total_lat.append(point[1])
    return all_centralpoint, new_total_lon,new_total_lat

## 对三块区域均进行粗分域

In [ ]:
for i in range(len(polygon_cdnts)):
    tlon, tlat, tidx = div_board(polygon_lat[i], polygon_lon[i])

## 草稿

In [ ]:
matplt.figure(dpi=200)
matplt.plot(tlon, tlat, lw=0.1)
matplt.show()